In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

In [26]:
url = 'https://www.tomshardware.com/reviews/gpu-hierarchy,4388.html'
headers = {
    "user-Agent": "Mozilla/5.0"
}

In [27]:
response = requests.get(url, headers=headers)
response

<Response [200]>

In [28]:
soup = BeautifulSoup(response.text, "html.parser")
soup

<!DOCTYPE html>

<html class="tomshardware" data-locale="US" dir="ltr" lang="en">
<head>
<!-- [METATAGS - critical] -->
<meta charset="utf-8"/>
<meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
<title>GPU Benchmarks Hierarchy 2025 - Graphics Card Rankings | Tom's Hardware</title>
<meta content="2025-04-17T16:14:08+00:00" name="pub_date"/>
<meta content="Our GPU benchmarks hierarchy uses performance testing to rank all the current and previous generation graphics cards, showing how old and new GPUs stack up." name="description"/>
<link href="https://www.tomshardware.com/reviews/gpu-hierarchy,4388.html" rel="canonical"/>
<link crossorigin="" href="https://vanilla.futurecdn.net/tomshardware" rel="preconnect"/>
<link href="https://bordeaux.futurecdn.net" rel="preconnect"/>
<link href="https://cdn.mos.cms.futurecdn.net" rel="preconnect"/>
<link href="https://vanilla.futurecdn.net" rel="preconnect"/>
<link href="https://bordeaux.futurecdn.net" rel="dns-pref

In [30]:
tables = soup.find_all("table")
print(f'Found {len(tables)} tables')

Found 3 tables


In [31]:
target_table = tables[1]
target_table

<table class="table__wrapper table__wrapper--inbodyContent table__wrapper--sticky table__wrapper--divider" tabindex="0"><caption class="table__caption table__caption--top table__caption--left">GPU Benchmarks Hierarchy — Ray Tracing Performance</caption><thead class="table__head"><tr class="table__head__row"><th class="table__head__heading table__head__heading--left" colspan="1">Graphics Card</th><th class="table__head__heading table__head__heading--left" colspan="1">Lowest Price</th><th class="table__head__heading table__head__heading--left" colspan="1">MSRP</th><th class="table__head__heading table__head__heading--left" colspan="1">1080p Medium</th><th class="table__head__heading table__head__heading--left" colspan="1">1080p Ultra</th><th class="table__head__heading table__head__heading--left" colspan="1">1440p Ultra</th><th class="table__head__heading table__head__heading--left" colspan="1">4K Ultra</th><th class="table__head__heading table__head__heading--left" colspan="1">Specifica

In [32]:
headers = [th.text.strip() for th in target_table.find_all("th")]
headers

['Graphics Card',
 'Lowest Price',
 'MSRP',
 '1080p Medium',
 '1080p Ultra',
 '1440p Ultra',
 '4K Ultra',
 'Specifications (Links to Review)']

In [34]:
rows = []
for tr in target_table.find_all("tr")[1:]:
    cols = [td.text.strip() for td in tr.find_all("td")]
    if cols:
        rows.append(cols)
rows

[['GeForce RTX 5090',
  '$3,680',
  '$2,000',
  '100.0% (200.3)',
  '100.0% (154.6)',
  '100.0% (137.7)',
  '100.0% (89.6)',
  'GB202, 21760 shaders, 2407MHz, 32GB GDDR7@28Gbps, 1792GB/s, 575W'],
 ['GeForce RTX 4090',
  '$2,925',
  '$1,600',
  '91.9% (184.1)',
  '92.3% (142.7)',
  '85.5% (117.7)',
  '78.8% (70.6)',
  'AD102, 16384 shaders, 2520MHz, 24GB GDDR6X@21Gbps, 1008GB/s, 450W'],
 ['GeForce RTX 5080',
  '$1,449',
  '$1,000',
  '86.2% (172.6)',
  '84.9% (131.2)',
  '73.0% (100.5)',
  '63.7% (57.0)',
  'GB203, 10752 shaders, 2617MHz, 16GB GDDR7@30Gbps, 690GB/s, 360W'],
 ['GeForce RTX 4080 Super',
  '$1,230',
  '$1,000',
  '83.2% (166.6)',
  '80.5% (124.4)',
  '68.5% (94.3)',
  '60.7% (54.4)',
  'AD103, 10240 shaders, 2550MHz, 16GB GDDR6X@23Gbps, 736GB/s, 320W'],
 ['GeForce RTX 5070 Ti',
  '$980',
  '$750',
  '80.7% (161.6)',
  '77.9% (120.4)',
  '64.3% (88.5)',
  '54.5% (48.8)',
  'GB203, 8960 shaders, 2452MHz, 16GB GDDR7@28Gbps, 896GB/s, 300W'],
 ['Radeon RX 9070 XT',
  '$1,070',


In [35]:
df = pd.DataFrame(rows, columns=headers)
df

,Graphics Card,Lowest Price,MSRP,1080p Medium,1080p Ultra,1440p Ultra,4K Ultra,Specifications (Links to Review)
0,GeForce RTX 5090,"$3,680","$2,000",100.0% (200.3),100.0% (154.6),100.0% (137.7),100.0% (89.6),"GB202, 21760 shaders, 2407MHz, 32GB GDDR7@28Gb..."
1,GeForce RTX 4090,"$2,925","$1,600",91.9% (184.1),92.3% (142.7),85.5% (117.7),78.8% (70.6),"AD102, 16384 shaders, 2520MHz, 24GB GDDR6X@21G..."
2,GeForce RTX 5080,"$1,449","$1,000",86.2% (172.6),84.9% (131.2),73.0% (100.5),63.7% (57.0),"GB203, 10752 shaders, 2617MHz, 16GB GDDR7@30Gb..."
3,GeForce RTX 4080 Super,"$1,230","$1,000",83.2% (166.6),80.5% (124.4),68.5% (94.3),60.7% (54.4),"AD103, 10240 shaders, 2550MHz, 16GB GDDR6X@23G..."
4,GeForce RTX 5070 Ti,$980,$750,80.7% (161.6),77.9% (120.4),64.3% (88.5),54.5% (48.8),"GB203, 8960 shaders, 2452MHz, 16GB GDDR7@28Gbp..."
5,Radeon RX 9070 XT,"$1,070",$600,76.9% (154.0),72.0% (111.2),58.7% (80.9),50.0% (44.8),"Navi 48, 4096 shaders, 2970MHz, 16GB GDDR6@20G..."
6,GeForce RTX 4070 Ti Super,"$1,149",$800,76.7% (153.5),72.3% (111.8),57.7% (79.5),50.4% (45.1),"AD103, 8448 shaders, 2610MHz, 16GB GDDR6X@21Gb..."
7,GeForce RTX 4070 Ti,$900,$800,73.4% (147.0),69.5% (107.4),54.4% (74.9),44.6% (40.0),"AD104, 7680 shaders, 2610MHz, 12GB GDDR6X@21Gb..."
8,Radeon RX 7900 XTX,"$1,120","$1,000",70.0% (140.2),66.4% (102.6),54.0% (74.3),44.9% (40.2),"Navi 31, 6144 shaders, 2500MHz, 24GB GDDR6@20G..."
9,Radeon RX 9070,$670,$550,69.7% (139.6),63.7% (98.5),50.8% (69.9),42.0% (37.7),"Navi 48, 3584 shaders, 2520MHz, 16GB GDDR6@20G..."


In [ ]:
load_dotenv()

pg_user = os.getenv('PG_USER')
pg_password = os.getenv('PG_PASSWORD')
pg_host = os.getenv('PG_HOST')
pg_port = os.getenv('PG_PORT')
pg_db = os.getenv('PG_GPU_DB')

pg_conn_str = f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}"
pg_engine = create_engine(pg_conn_str)

In [ ]:
with pg_engine.begin() as conn:
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS sql_project;"))

In [40]:
df.to_sql("gpu_hierarchy_web_raw", con=pg_engine, schema="sql_project", if_exists="replace", index=False)

28